In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_06_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-10 21:15:30.836380


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_06_02/m_06_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_06_02/06_2a_posner_task_2023-05-18_10h59.43.934/06_2a_posner_task_2023-05-18_10h59.43.934.csv
Found '2b' in file: m_06_02/06_2b_posner_task_2023-05-18_12h04.59.098/06_2b_posner_task_2023-05-18_12h04.59.098.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
len(events) 413
len(validity) 100
# of stim events 100
[    0     0 99999]
[12473     0    40]
[13973     0    70]
[17093     0    50]
[17193     0    51]
[18973     0    40]
[20473     0    70]
[23003     0    50]
[23103     0    51]
[25473     0    40]
[26973     0    80]
[30783   

/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [379] Keeping  (2023-05-18 11:12:52.535782+00:00 - 2023-05-18 11:12:52.535782+00:00 -> 653.167 - 653.167)
  [380] Keeping  (2023-05-18 11:12:52.635782+00:00 - 2023-05-18 11:12:52.635782+00:00 -> 653.267 - 653.267)
  [381] Keeping  (2023-05-18 11:12:53.995782+00:00 - 2023-05-18 11:12:53.995782+00:00 -> 654.627 - 654.627)
  [382] Keeping  (2023-05-18 11:12:55.495782+00:00 - 2023-05-18 11:12:55.495782+00:00 -> 656.127 - 656.127)
  [383] Keeping  (2023-05-18 11:12:58.165782+00:00 - 2023-05-18 11:12:58.165782+00:00 -> 658.797 - 658.797)
  [384] Keeping  (2023-05-18 11:12:58.265782+00:00 - 2023-05-18 11:12:58.265782+00:00 -> 658.897 - 658.897)
  [385] Keeping  (2023-05-18 11:13:00.495782+00:00 - 2023-05-18 11:13:00.495782+00:00 -> 661.127 - 661.127)
  [386] Keeping  (2023-05-18 11:13:01.995782+00:00 - 2023-05-18 11:13:01.995782+00:00 -> 662.627 - 662.627)
  [387] Keeping  (2023-05-18 11:13:05.095782+00:00 - 2023-05-18 11:13:05.095782+00:00 -> 665.727 - 665.727)
  [388] Keeping  (2023-05-18

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...
5 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 35.3s.


Method,fastica
Fit,52 iterations on raw data (690203 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=690203
    Range : 12473 ... 702675 =     12.473 ...   702.675 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
345 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
345 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
345 matching events found
No baseline correction applied
0 projection items activated


In [7]:
print(ica.exclude)

[0]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 1 ICA component
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_06_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/cleaned/m_06_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_06_02/m_06_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_06_02
├── m_06_02_pos2a.eeg
├── m_06_02_pos2a.vhdr
└── m_06_02_pos2a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_06_02/m_06_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '2a' in file: m_06_02/06_2a_posner_task_2023-05-18_10h59.43.934/06_2a_posner_task_2023-05-18_10h59.43.934.csv
Found '2b' in file: m_06_02/06_2b_posner_task_2023-05-18_12h04.59.098/06_2b_posner_task_2023-05-18_12h04.59.098.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']


/Users/babe/src/gla/dissertation/go/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


len(events) 426
len(validity) 100
# of stim events 102
[    0     0 99999]
[40254     0    10]
[76936     0    40]
[78436     0    70]
[81906     0    50]
[82006     0    51]
[83436     0    40]
[84936     0    70]
[88776     0    50]
[88876     0    51]
[89936     0    40]
[91436     0    60]
[326345      0     10]
[358567      0     40]
[360067      0     60]
[363617      0     50]
[363717      0     51]
[365067      0     40]
[366567      0     80]
[370077      0     50]
[370177      0     51]
[371568      0     40]
[373067      0     80]
[376347      0     50]
[376447      0     51]
[378068      0     40]
[379567      0     70]
[383347      0     50]
[383447      0     51]
[384568      0     40]
[386067      0     70]
[389287      0     50]
[389387      0     51]
[391068      0     40]
[392567      0     80]
[396258      0     50]
[396358      0     51]
[397568      0     40]
[399068      0     80]
[402428      0     50]
[402527      0     51]
[404078      0     40]
[405578      0 

IndexError: index 100 is out of bounds for axis 0 with size 100

In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
415 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 415 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [7]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-04-05 16:11:31.870965+00:00 - 2023-04-05 16:22:52.180965+00:00
  [0] Keeping  (2023-04-05 16:11:31.870965+00:00 - 2023-04-05 16:11:31.870965+00:00 -> 107.16 - 107.16)
  [1] Keeping  (2023-04-05 16:11:33.370965+00:00 - 2023-04-05 16:11:33.370965+00:00 -> 108.66 - 108.66)
  [2] Keeping  (2023-04-05 16:11:36.550965+00:00 - 2023-04-05 16:11:36.550965+00:00 -> 111.84 - 111.84)
  [3] Keeping  (2023-04-05 16:11:36.650965+00:00 - 2023-04-05 16:11:36.650965+00:00 -> 111.94 - 111.94)
  [4] Keeping  (2023-04-05 16:11:38.377965+00:00 - 2023-04-05 16:11:38.377965+00:00 -> 113.667 - 113.667)
  [5] Keeping  (2023-04-05 16:11:39.880965+00:00 - 2023-04-05 16:11:39.880965+00:00 -> 115.17 - 115.17)
  [6] Keeping  (2023-04-05 16:11:43.520965+00:00 - 2023-04-05 16:11:43.520965+00:00 -> 118.81 - 118.81)
  [7] Keeping  (2023-04-05 16:11:43.620965+00:00 - 2023-04-05 16:

/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


  [146] Keeping  (2023-04-05 16:15:32.621965+00:00 - 2023-04-05 16:15:32.621965+00:00 -> 347.911 - 347.911)
  [147] Keeping  (2023-04-05 16:15:34.641965+00:00 - 2023-04-05 16:15:34.641965+00:00 -> 349.931 - 349.931)
  [148] Keeping  (2023-04-05 16:15:36.141965+00:00 - 2023-04-05 16:15:36.141965+00:00 -> 351.431 - 351.431)
  [149] Keeping  (2023-04-05 16:15:38.801965+00:00 - 2023-04-05 16:15:38.801965+00:00 -> 354.091 - 354.091)
  [150] Keeping  (2023-04-05 16:15:38.902965+00:00 - 2023-04-05 16:15:38.902965+00:00 -> 354.192 - 354.192)
  [151] Keeping  (2023-04-05 16:15:41.141965+00:00 - 2023-04-05 16:15:41.141965+00:00 -> 356.431 - 356.431)
  [152] Keeping  (2023-04-05 16:15:42.641965+00:00 - 2023-04-05 16:15:42.641965+00:00 -> 357.931 - 357.931)
  [153] Keeping  (2023-04-05 16:15:45.141965+00:00 - 2023-04-05 16:15:45.141965+00:00 -> 360.431 - 360.431)
  [154] Keeping  (2023-04-05 16:15:45.241965+00:00 - 2023-04-05 16:15:45.241965+00:00 -> 360.531 - 360.531)
  [155] Keeping  (2023-04-05

Method,fastica
Fit,1000 iterations on raw data (680310 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=680310
    Range : 107160 ... 787469 =    107.160 ...   787.469 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items ac

In [9]:
ica_b.exclude

[0, 1, 2]

In [10]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 3 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [15]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/cleaned/m_00_01/m_00_01_pos1b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [22]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/cleaned/m_00_01
├── m_00_01_pos1a.eeg
├── m_00_01_pos1a.vhdr
├── m_00_01_pos1a.vmrk
├── m_00_01_pos1b.eeg
├── m_00_01_pos1b.vhdr
└── m_00_01_pos1b.vmrk

1 directory, 6 files


# ...

In [16]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b